In [1]:
from cellitaire.environment.agents.PPOAgent import Agent
from cellitaire.environment.cellitaire_env import CellitaireEnv
from cellitaire.environment.rewards.reward import *
from cellitaire.environment.rewards.foundation_rewards import *

board_rows = 7
board_cols = 12
num_reserved = 6
test_reward = CombinedReward([
    #PlacedCardInFoundationReward(weight=6),
    WinReward(weight=100, rows=board_rows, cols=board_cols),
    #ConstantReward(weight=0.5),
    ScalingPlacedCardInFoundationReward(weight=1, rows=board_rows, cols=board_cols),
    #PlayedLegalMoveReward(weight=1, rows=board_rows, cols=board_cols, num_reserved = num_reserved),
    #PeriodicPlacedCardInFoundationReward(weight=4, reward_period=3),
    #CreatedMovesReward(weight=1, num_reserved=num_reserved, foundation_count_dropoff=30)
])
N = 10000
batch_size = 2000
n_epochs = 5
alpha = 4e-6
gamma = 0.99
gae_lambda = 0.95
checkpoint_dir = 'tmp/testing_parallel_2'


pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import time
import numpy as np

try:
    best_score = -1000
    score_history = []
    max_score = 0
    episodes_without_best = 0
    
    demo_env = CellitaireEnv(test_reward, rows=board_rows, cols=board_cols, num_reserved=num_reserved, max_moves=1200, max_illegal_moves=1200, render_mode='human', frame_rate=0.1)
    demo_env.render()
    
    demo_agent = Agent(n_actions=demo_env.action_space.n, 
        input_dims=(board_rows * board_cols * 4 + 6,), batch_size=batch_size, fc1_actor=2048, fc2_actor=2048, fc1_critic=4096, fc2_critic=4096,
                    alpha=alpha, n_epochs=n_epochs, gamma=gamma, gae_lambda=gae_lambda, checkpoint_dir=checkpoint_dir)
    demo_agent.load_models()
    
    learn_iters = 0
    avg_score = 0
    n_steps = 0
    i = 0
    while True:
        demo_agent.load_models()
        demo_env.reset()
        observation = demo_env.get_state()
        done = False
        score = 0
        while not done:
            action, _, _ = demo_agent.choose_legal_action_mostly(observation, demo_env.get_legal_actions_as_int())
            observation_, reward, done, _, info = demo_env.step(action)
            n_steps += 1
            score += reward
            observation = observation_
        i += 1
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        
        max_score = max(max_score, score)
        
        if avg_score > best_score and i > N:
            best_score = avg_score
            recent_std = np.std(score_history[-100:])
            #agent.save_models()
            episodes_without_best = 0
            print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done} *')
        else:
            episodes_without_best += 1
            
        if episodes_without_best % 100 == 0 and episodes_without_best > 0:
            recent_std = np.std(score_history[-100:])
            print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done}')
        time.sleep(8)
        print(score)
except:
    demo_env.close()

820.0
78.0
153.0
253.0
1128.0
66.0
528.0
595.0
666.0
780.0
780.0
120.0
703.0
153.0
820.0
990.0
561.0
595.0
630.0
561.0
1035.0
990.0
741.0
703.0
946.0
630.0
780.0
231.0
105.0
300.0
561.0
496.0
351.0
66.0
1.0
741.0
120.0
120.0
120.0
703.0
136.0
820.0
861.0
595.0
406.0
703.0
741.0
561.0
780.0
171.0
105.0
406.0
496.0
861.0
561.0
91.0
903.0
6.0
3.0
630.0
1035.0
666.0
378.0
435.0
276.0
703.0
496.0
666.0
36.0
666.0
1081.0
666.0
28.0
105.0
946.0
946.0
1.0
861.0
703.0
91.0
820.0
496.0
351.0
3.0
861.0
190.0
3.0
861.0
528.0
741.0
820.0
741.0
780.0
666.0
630.0
3.0
45.0
0
820.0
episode   100 | score   10.0 | avg  507.1 | std 327.07 | max score 1128.0 | learning steps     0 | done True
10.0
28.0
3.0
153.0
595.0
276.0
15.0
595.0
378.0
136.0
21.0
406.0
406.0
703.0
703.0
820.0
66.0
276.0
153.0
630.0
1081.0
0
595.0
528.0
15.0
28.0
435.0
780.0
1128.0
136.0
300.0
28.0
946.0
3.0
990.0
1081.0
1176.0
990.0
465.0
465.0
946.0
15.0
231.0
741.0
630.0
903.0
105.0
15.0
435.0
561.0
528.0
496.0
91.0
15.0
946.0
351.0

In [ ]:
demo_env.close()